In [ ]:
# First of all, you need to import OACIS module. Add the OACIS_ROOT path to sys.path to import `oacis` module
import os,sys
sys.path.append( os.environ['OACIS_ROOT'] )
import oacis

# Prerequisites

Before running the following samples, please prepare a sample simulator as follows.

    Name: sample_simulator
    Parameter definitions:
        "p1", Float, 1.0
        "p2", Float, 2.0
        "p3", Float, 3.0
    Command: ruby -r json -e 'j = JSON.load(File.read("_input.json")); sum=%w(p1 p2 p3).map{|k| j[k]}.inject(:+); puts({sum: sum}.to_json)' > _output.json
    Input type: JSON
    Executable_on : localhost


In [ ]:
if oacis.Simulator.where(name="sample_simulator").size() == 0:
    sim = {
        "name": "sample_simulator",
        "command": "ruby -r json -e 'j = JSON.load(File.read(\"_input.json\")); sum=%w(p1 p2 p3).map{|k| j[k]}.inject(:+); puts({sum: sum}.to_json)' > _output.json",
        "support_input_json": True,
        "parameter_definitions": [
            {"key": "p1", "type": "Float", "default": 1.0},
            {"key": "p2", "type": "Float", "default": 2.0},
            {"key": "p3", "type": "Float", "default": 3.0}
          ],
          "executable_on": [ oacis.Host.find_by_name("localhost")]
        }
    oacis.Simulator.create(sim)

# Creating parameter sets and runs


## Retrieving / Creating a simulator
First let's check for the existence and if necessary create a simulator.

In [ ]:
sim = oacis.Simulator.find_by_name("sample_simulator")
ps = sim.find_or_create_parameter_set( {"p1":10.0,"p2":20.0,"p3":30.0} )
ps.inspect()

## Creating runs

When creating runs, you need to specify the host to which the jobs are submitted to. We may need to set "host_parameters" which specify the parameters required by job schedulers.

In [ ]:

host = oacis.Host.find_by_name("localhost")
new_runs = ps.find_or_create_runs_upto( 5, submitted_to= host )
new_run_ids = [ str( run.id() ) for run in new_runs ]
new_run_ids


## Creating multiple parameter sets

In [ ]:
import numpy as np
sim = oacis.Simulator.where( name = 'sample_simulator' ).first()
p1_values = np.linspace(1.0,5.0, 5)
p2_values = np.linspace(2.0, 20.0, 10)
base_param = sim.default_parameters()
print(base_param)

param = base_param

for p1 in p1_values:
    for p2 in p2_values:
        param.update( {'p1':p1, 'p2':p2} )
        print(param)
        ps = sim.find_or_create_parameter_set( param )
        runs = ps.find_or_create_runs_upto(1, submitted_to=host)
        print( [ run.id().to_s() for run in runs] )
        
print(base_param)

# Obtaining execution results

For the sake of this example, we are going to investigate the dependency of "p1" and "p2" against the results.
Note that the code below should be run **after** the runs/jobs created above have completed their execution.

In [ ]:
sim = oacis.Simulator.find_by_name('sample_simulator')
print(f"Simulation name {sim.name()}")
print(f"Command {sim.command()}")
print("Executable on:")
for h in sim.executable_on():
    print(f"\t{h.name()}")

print(f"Total    runs: {len( sim.runs() )}")
print(f"Finished runs: {len(sim.runs().where({'status':'finished'}))}" )

In [ ]:
# ParameterSet of {p1: 1.0, p2: 2.0, p3: 3.0}
ps = sim.find_parameter_set( {'p1':1.0, 'p2':2.0, 'p3':3.0} )

#  Here are some examples of fields of ParameterSet
print( ps.id(), ps.v(), ps.runs().count(), ps.runs().where({'status':'finished'}).count() )
#print( ps.runs_status_count() )
print( ps.average_result("sum") )

In [ ]:
# Getting a Run
run = ps.runs().first()

# Here are some examples of fields of Run
for field in [ run.id(), run.status(), run.result(), run.cpu_time(), run.finished_at(), run.dir() ]:
    print( field )


## Search parameter sets by condition

In [ ]:
# Search ParameterSets by condition
sim = oacis.Simulator.where(name="sample_simulator").first()

# The followings are some example queries which you might find useful.
# Since a ParameterSet contains the parameters in the field of `v`, you need to issue an query against `v`.
# See http://www.rubydoc.info/github/mongoid/origin/Origin/Selectable
print("p2 == 10.0 and p3 == 3.0")
found = sim.parameter_sets().where({"v.p2":10.0, "v.p3":3.0}).asc("v.p1")
for ps in found:
    print( ps.v() )

print("p1 <= 2 and p2 > 15.0 and p3 == 3.0")
found = sim.parameter_sets().lte( {"v.p1":2} ).gt( {"v.p2":15.0} ).where( {"v.p3":3.0} )
for ps in found:
    print( ps.v() )

print("p1 is in [2,4] and p2==20")
found = sim.parameter_sets().send('in', {"v.p1":[2,4]} ).where( {"v.p2":20} )
for ps in found:
    print( ps.v() )

print("p1 <= 3 and p2 <= 6.0, ordered by p1 in ascending order and p2 in descending order")
found = sim.parameter_sets().lte( {"v.p1":3, "v.p2": 6.0} ).order_by( {"v.p1":'asc', "v.p2":'desc'} )
for ps in found:
    print( ps.v() )


## Plot results using matplotlib

Since we are in jupyterlab, we might as well use the integration of all the most practical features to obtain the information we want.

In [ ]:
# plot results using matplotlib
import matplotlib.pyplot as plt

sim = oacis.Simulator.where(name='sample_simulator').first()
parameter_sets = sim.parameter_sets().where( {"v.p1":1.0, "v.p3":3.0} ).asc("v.p2")
x = []
y = []
for ps in parameter_sets:
    x.append( ps.v()["p2"] )
    y.append( ps.average_result("sum")[0] )
plt.plot(x,y, 'ro')
plt.show()

# Deleting things

The `discard` method can be used to delete an item. After running the code below, you can check on the OACIS page that the runs in question have indeed been deleted.

In [ ]:
sim = oacis.Simulator.where(name="sample_simulator").first()
ps = sim.find_parameter_set( {"p1":1.0, "p2": 2.0, "p3": 3.0} )
print( len( ps.runs() ) )
run = ps.runs()[0]                                            # Get the first run
run.discard()                                                     # Delete it
print( len(ps.runs()) )                                      # => 4.  The number of runs is decreased by 1.


#  deleting all ParameterSet whose p2 is larger than 10.0
print( sim.parameter_sets().gt( {"v.p2": 10.0} ).count() > 0 )            # => true
for ps in sim.parameter_sets().gt( {"v.p2": 10.0} ):
  ps.discard()
print( sim.parameter_sets().gt( {"v.p2": 10.0} ).count() > 0 )            # => false